# Larger systems

```{note}
- [ ] Consider what should be included
- [ ] Start working on it
    - Might include examples which are not executable
```

Considering larger systems and more.

## Valence-to-core XES

Suitable TM-complex with minimal static correlation

1. Overlap

2. TDDFT(?)

3. ADC?


## XAS of uracil

Maybe uracil or similar:

1. Tailor CVS space for ADC ('cost-saving measures')

2. Gas phase spectra

3. Spectra with PE

Might include massive system with DFT as well.


## XES of uracil

Maybe uracil again.

1. Gas phase

2. Environment with PE

Could potentially consider some massive system - TDDFT versus overlap.


## Coupling to molecular dynamics

Some system...

1. Dynamics + XAS

2. Dynamics + XES

3. Inclusion of environment?

4. Core-hole dynamics with some model?

## Transient X-ray spectroscopy

```{note}
Longer-term goal.

- Two-step approach

- Coupling
```